In [103]:
# Librerías necesarias
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import json
import random
import time
from datetime import datetime
import datetime
import os
import numpy as np 
from millify import millify,prettify
from tqdm import tqdm

import credential_yt

In [104]:
#DEVELOPER_KEY = cred.API_KEY
DEVELOPER_KEY = credential_yt.API_KEY

## Estadísticas del canal de YT - FC Barcelona

In [105]:
url = "https://www.youtube.com/channel/UC14UlmYlSNiQCBe9Eookf_A"

In [106]:
CANAL_ID = 'UC14UlmYlSNiQCBe9Eookf_A'

url_canal_estadisticas = 'https://youtube.googleapis.com/youtube/v3/channels?part=statistics&id='+CANAL_ID+'&key='+DEVELOPER_KEY
estadisticas_canal = requests.get(url_canal_estadisticas).json()
estadisticas_canal

{'kind': 'youtube#channelListResponse',
 'etag': 'L6iRZGBzEyy8PSJHZ7OZXUsvqnM',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'jicXFD09DiF1sJoXAos6Rq_QR-8',
   'id': 'UC14UlmYlSNiQCBe9Eookf_A',
   'statistics': {'viewCount': '2911840648',
    'subscriberCount': '15900000',
    'hiddenSubscriberCount': False,
    'videoCount': '11261'}}]}

In [107]:
# Extraemos la información de las estadisticas
estadisticas_canal=estadisticas_canal['items'][0]['statistics']
# Definimos la fecha de creación
date = pd.to_datetime('today').strftime('%Y-%m-%d')

In [108]:
# Definimos un diccionario con la información que necesitamos
datos_canal = {
    'Create_at':date,
    'Total_Vistas':int(float(estadisticas_canal['viewCount'])),
    'Suscripciones':int(float(estadisticas_canal['subscriberCount'])),
    'Cantidad_Videos':int(float(estadisticas_canal['videoCount']))
}
datos_canal

{'Create_at': '2023-10-18',
 'Total_Vistas': 2911840648,
 'Suscripciones': 15900000,
 'Cantidad_Videos': 11261}

### Funciones

In [109]:
# Función para retornar las estadísticas 
def get_estadisticas(api_key, canal_id):
    
    url_canal_estadisticas = 'https://youtube.googleapis.com/youtube/v3/channels?part=statistics&id='+canal_id+'&key='+api_key
    estadisticas_canal = requests.get(url_canal_estadisticas).json()
    
    estadisticas_canal=estadisticas_canal['items'][0]['statistics']
    date = pd.to_datetime('today').strftime('%Y-%m-%d')
    datos_canal = {
    'Create_at':date,
    'Total_Vistas':int(float(estadisticas_canal['viewCount'])),
    'Suscripciones':int(float(estadisticas_canal['subscriberCount'])),
    'Cantidad_Videos':int(float(estadisticas_canal['videoCount']))
    }
    
    return datos_canal

In [110]:
# Probamos la función
get_estadisticas(DEVELOPER_KEY, CANAL_ID)

{'Create_at': '2023-10-18',
 'Total_Vistas': 2911840648,
 'Suscripciones': 15900000,
 'Cantidad_Videos': 11261}

### Análisis de Canales

In [111]:
nombre_canales  =  ['Fc Barcelona','Real Madrid','Atletico Madrid','Sevilla']
canales_id  =  ['UC14UlmYlSNiQCBe9Eookf_A','UCWV3obpZVGgJ3j9FVhEjF2Q','UCuzKFwdh7z2GHcIOX_tXgxA','UCLy9lmj_0cqffXUzbGHNmYA']

In [112]:
# Lo unimos en un DF
canales = {
    'Nombre_Canal':nombre_canales,
    'Id_Canal':canales_id}

df_canales = pd.DataFrame(canales)

In [113]:
# Visualizamos el DF
df_canales

,Nombre_Canal,Id_Canal
0,Fc Barcelona,UC14UlmYlSNiQCBe9Eookf_A
1,Real Madrid,UCWV3obpZVGgJ3j9FVhEjF2Q
2,Atletico Madrid,UCuzKFwdh7z2GHcIOX_tXgxA
3,Sevilla,UCLy9lmj_0cqffXUzbGHNmYA


In [114]:
# Probamos la función con información del DF
get_estadisticas(DEVELOPER_KEY,df_canales['Id_Canal'][0])

{'Create_at': '2023-10-18',
 'Total_Vistas': 2911840648,
 'Suscripciones': 15900000,
 'Cantidad_Videos': 11261}

### Función para iterar en cada uno de los canales

In [115]:
def canal_estadisticas(df,api_key):
    
    fecha = []
    vistas = []
    suscripciones = []
    cantidad_videos = []
    nombre_canal = []
    
    tiempo = [1,2.5,3,2]
    
    # Iterar el DF
    for i in tqdm(range(len(df)),colour = 'blue'):
        
        stats_temp = get_estadisticas(DEVELOPER_KEY,df_canales['Id_Canal'][i])
        
        fecha.append(stats_temp['Create_at'])
        vistas.append(stats_temp['Total_Vistas'])
        suscripciones.append(stats_temp['Suscripciones'])
        cantidad_videos.append(stats_temp['Cantidad_Videos'])
        nombre_canal.append(df['Nombre_Canal'][i])
        
        time.sleep(random.choice(tiempo))
    
    datos = {
        
        'Nombre_Canal':nombre_canal,
        'Suscripciones':suscripciones,
        'Cantidad_Videos':cantidad_videos,
        'Total_Vistas':vistas,
        'created_at':fecha,
        
    }
    
    df_canales_final = pd.DataFrame(datos)
    
    return df_canales_final     

In [116]:
# Probamos la función
canal_estadisticas(df_canales, DEVELOPER_KEY)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.77s/it]


,Nombre_Canal,Suscripciones,Cantidad_Videos,Total_Vistas,created_at
0,Fc Barcelona,15900000,11261,2911840648,2023-10-18
1,Real Madrid,11300000,6517,1728292469,2023-10-18
2,Atletico Madrid,349000,2950,39843286,2023-10-18
3,Sevilla,221000,13573,68875874,2023-10-18


In [117]:
df_canales

,Nombre_Canal,Id_Canal
0,Fc Barcelona,UC14UlmYlSNiQCBe9Eookf_A
1,Real Madrid,UCWV3obpZVGgJ3j9FVhEjF2Q
2,Atletico Madrid,UCuzKFwdh7z2GHcIOX_tXgxA
3,Sevilla,UCLy9lmj_0cqffXUzbGHNmYA


In [122]:
# Descargar el archivo en formato CSV

current_datetime = datetime.now()
year_month = current_datetime.strftime('%Y%m%d')
file_name = f'./input/canal_estadisticas_{year_month}.csv'

df_canales.to_csv(file_name, index=False)